In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image,ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm.notebook import tqdm
import numpy as np

# Loading Dataset

In [2]:
from pathlib import Path
import requests
import subprocess
URL = "https://storage.googleapis.com/kaggle-data-sets/531544/973292/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1585134065&Signature=eZK7Px2EobN2PI61sybJ9crRRrgcu3RXJvjZOAKPkHoCdcbYnxYOWnPngYzfn6rinhIsArZReWDug8PaLaFv5nZRJ%2Fbv%2Bh2PWJV795cb%2F5J6fVRzCsc7cqu0ojcB%2BjKnoRK6KprK7bU6lg4YIj10cL1Wf%2FggmtgASkxiK3DBCOnAuj3l6%2BplPI%2B1Cpnhlpov07gewbnGhRhI1eCVGgeaj5hBxT%2F8NEvZ73OgztLsRLiKFDNIHm9PZ3RnanLIejVr7eKmDd0Er9pyMqir0wftgsCsh%2FW%2F1cknCSGeZhZOsZB%2BO1oUox3yLLsVRLRi384UCpuvE6g7b3XLiGR%2FGsCtpA%3D%3D&response-content-disposition=attachment%3B+filename%3Dmemotion-dataset-7k.zip"

DATA_PATH = Path("data")
PATH = DATA_PATH

PATH.mkdir(parents=True, exist_ok=True)

# URL = "http://deeplearning.net/data/mnist/"

FILENAME = "memotion-dataset.zip"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

subprocess.call(["unzip", "-qq", PATH/FILENAME, "-d", DATA_PATH])

1

In [0]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd

class MemotionDatasetImage(Dataset):
    def __init__(self, pickle_file, image_dir, transform=None):
        self.df = pd.read_pickle(pickle_file)
        self.image_dir = image_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(self.image_dir+'/'+row.image_name).convert('RGB')
        if self.transform:
            tensor = self.transform(image)
        else:
          tensor = transform.toTensor(image)
        return tensor

In [0]:
dataset = MemotionDatasetImage(
    'data/memotion_dataset_7k/labels_pd_pickle',
    'data/memotion_dataset_7k/images',
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]))

In [0]:
def extract_visual_features(model_name, layer_name, dataset):
    get_model = getattr(models, model_name)
    model = get_model(pretrained=True)
    model = model.cuda()
    model.eval()

    features = []
    def hook(module, input, output):
        N,C,H,W = output.shape
        output = output.reshape(N,C,-1)
        features.append(output.mean(dim=2).cpu().detach().numpy())
    handle = model._modules.get(layer_name).register_forward_hook(hook)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
    
    for i_batch, inputs in tqdm(enumerate(dataloader), total=len(dataloader)):
        _ = model(inputs.cuda())

    features = np.concatenate(features)

    features = pd.DataFrame(features)
    features = features.add_prefix('IMAGE_')
    features.loc[:,'image_name'] = dataset.df['image_name']
    
    handle.remove()
    del model

    return features

In [8]:
image_features = extract_visual_features('resnet18', 'avgpool', dataset)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [0]:
image_features.to_pickle('image_features_pd_pickle')